In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('paper_data.csv')

In [ ]:
df

,Unnamed: 0,PV generation,day_length,sunrise_seconds,sunset_seconds,hour_cos,hour_sin,solar_angle
0,0,0.797,10.298889,21477,58554,-0.500000,8.660254e-01,23.627011
1,1,1.155,10.298889,21477,58554,-0.707107,7.071068e-01,28.110852
2,2,1.357,10.298889,21477,58554,-0.866025,5.000000e-01,29.917599
3,3,1.873,10.298889,21477,58554,-0.965926,2.588190e-01,28.789834
4,4,2.180,10.298889,21477,58554,-1.000000,1.224647e-16,24.890537
...,...,...,...,...,...,...,...,...
11885,11885,5.103,11.233611,21501,61942,-0.965926,-2.588190e-01,27.387379
11886,11886,6.637,11.233611,21501,61942,-0.866025,-5.000000e-01,19.852171
11887,11887,4.219,11.233611,21501,61942,-0.707107,-7.071068e-01,11.013164
11888,11888,1.841,11.233611,21501,61942,-0.500000,-8.660254e-01,1.598756


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


features = df.drop(columns=['PV generation']).values
target = df['PV generation'].values


scaler_features = MinMaxScaler()
scaler_target = MinMaxScaler()

features_scaled = scaler_features.fit_transform(features)
target_scaled = scaler_target.fit_transform(target.reshape(-1, 1))


X_train, X_test, y_train, y_test = train_test_split(features_scaled, target_scaled, test_size=0.2, random_state=42)


gbr = GradientBoostingRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42
)


gbr.fit(X_train, y_train.ravel())


y_train_pred_scaled = gbr.predict(X_train)
y_test_pred_scaled = gbr.predict(X_test)


y_train_pred = scaler_target.inverse_transform(y_train_pred_scaled.reshape(-1, 1))
y_test_pred = scaler_target.inverse_transform(y_test_pred_scaled.reshape(-1, 1))
y_train_actual = scaler_target.inverse_transform(y_train)
y_test_actual = scaler_target.inverse_transform(y_test)


mse_train = mean_squared_error(y_train_actual, y_train_pred)
mae_train = mean_absolute_error(y_train_actual, y_train_pred)
r2_train = r2_score(y_train_actual, y_train_pred)


mse_test = mean_squared_error(y_test_actual, y_test_pred)
mae_test = mean_absolute_error(y_test_actual, y_test_pred)
r2_test = r2_score(y_test_actual, y_test_pred)


print("Training Metrics:")
print(f"MSE: {mse_train}, MAE: {mae_train}, R²: {r2_train}")
print("\nTesting Metrics:")
print(f"MSE: {mse_test}, MAE: {mae_test}, R²: {r2_test}")


results = pd.DataFrame({
    "Metric": ["MSE", "MAE", "R²"],
    "Train": [mse_train, mae_train, r2_train],
    "Test": [mse_test, mae_test, r2_test]
})

print("\nResults DataFrame:")
print(results)


Training Metrics:
MSE: 2.1279289141671844, MAE: 0.7914910335861051, R²: 0.7987722222891815

Testing Metrics:
MSE: 2.3386959882816236, MAE: 0.8211079325159231, R²: 0.784764491113423

Results DataFrame:
  Metric     Train      Test
0    MSE  2.127929  2.338696
1    MAE  0.791491  0.821108
2     R²  0.798772  0.784764
